<a id="section0"></a>
# Topics
* [Install nodeJS for Jupyter notebooks](#section1)
* [import NodeJS module](#section2)
* [First simple nodeJS example](#section3)
* [Functions](#section4)
    * [Global named functions](#section4a)
    * [Anonymous functions in Javascript](#section4b)

[back](#section0)
<a id="section1"></a>
## Install nodeJS for Jupyter notebooks
You need to install the following python modules. This will then allow you to run nodeJS from within a Jupyter notebook.
NOTE: This is a once only operation - you do NOT have to repeat this every time you use the notebook

See [&lt;here&gt;](https://medium.com/codait/nodebooks-node-js-data-science-notebooks-aa140bea21ba) for more details

In [ ]:
!pip install pixiedust
!pip install pixiedust_node

[back](#section0)
<a id="section2"></a>
## import NodeJS module
You need to do this everytime to start this notebook.
This will then allow you to create cells that start with %%node, that indicates they are nodeJS cells

In [1]:
import pixiedust_node

Pixiedust database opened successfully


pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.


[back](#section0)
<a id="section3"></a>
## First simple nodeJS example
* Creates a date and prints it out
* Then show that the variable is accessible into another cell

In [3]:
%%node
var date = new Date();
print(date);

"2020-05-09T07:56:38.142Z"


In [8]:
%%node
print(date)

"2020-05-09T07:56:38.142Z"


[back](#section0)
<a id="section4"></a>
## Functions
There are a number of ways to define functions in javascript

[back](#section0)
<a id="section4a"></a>
### Global function

In [19]:
%%node

function namedFunction (param1){
    print(param1)
}

... ...


In [20]:
%%node

namedFunction("Test it");


"Test it"


### Hiding functions in a closure
Which doesn't seem to work. The inner declared function is still available globally

In [24]:
%%node
function(){
function namedFunction2 (param1){
    print("Only available within this closure" + param1)
}
namedFunction2("namedFunction2 called ")
}();

Uncaught
... ...
"Only available within this closurenamedFunction2 called "
Uncaught


In [26]:
%%node
namedFunction2("namedFunction2 called ")

"Only available within this closurenamedFunction2 called "


[back](#section0)
<a id="section4b"></a>
### Anonymous functions in Javascript

#### Method 1

In [30]:
%%node
var func = function (msg){print(msg)}

func("Hello world!!")

"Hello world!!"


#### Method 2
Dropping the function keyword and using => instead

Anonymous functions created using "function" v "=>" are not exactly the same.
See [&lt;here&gt;](https://stackoverflow.com/questions/34361379/are-arrow-functions-and-functions-equivalent-exchangeable) for differences.

In [32]:
%%node
var func2 = (msg)=>{print(msg)}

func2("Hello world!!")

"Hello world!!"


[back](#section0)
<a id="section5"></a>
## Asynchronous v Synchronous
Each javascipt COMPUTE node (i.e. VM) is a single-thread application.
This means that any significant programs have to be written in an Asynchronous style.
    
An Asynchronous design allows for a single-threaded nodeJS program to have many tasks being performed 
at the same time without the need for multiple threads (like in other programming languages).

This approach has proved to be very successful and other languages (like python) are not adding asynchronous support into their language. The main reason for the success if that an asynchronous program tends to be a lot more scalable than one written in other languages (that utilises multiple threads).

The only issue is that writing and understanding asynchronous programs is harder than for synchronous programs.


### 3 Methods for defining Asynchronous programs in Javascript

#### Plain Javascript approach

In [40]:
%%node

function asyncFunc1(callback) {
    // console.log("asyncFunc1 start");
    // Do some work and then return result or error to the callback
    result = "Test result"
    callback(null,result)
    return result
}

... ... ... ... ... ...


In [46]:
%%node
//Use setTimeout to call the function asynchronously
setTimeout(function(){asyncFunc1((err,data)=>{print("received " + data)})}, 2000)
console.log("Here 2");

Here 2
"received Test result"


#### Javascript Promises

In [5]:
%%node

const promiseA = new Promise( (resolutionFunc,rejectionFunc) => {
    resolutionFunc(777);
});

// At this point, "promiseA" is already settled.
promiseA.then( (val) => console.log("asynchronous logging has val:",val) );
promiseA.then( (val) => console.log("asynchronous logging has val2222:",val) );
console.log("immediate logging");
console.log("immediate logging2");
console.log("immediate logging3");

... ... Uncaught
immediate logging
immediate logging2
immediate logging3
asynchronous logging has val: 777
asynchronous logging has val2222: 777


In [8]:
%%node

promiseB = new Promise( (resolutionFunc,rejectionFunc) => {
    try{
        throw "333";
        //resolutionFunc(111);
        //rejectionFunc(888)
    }
    catch (err){
        //console.log("In catch")
        rejectionFunc("Exception"+err);
    }
                             
});

// It looks like you only get one catch with a promise
promiseB.then( (val) => console.log("Resolved:",val)).catch((val) => console.log("Exception caught",val)).catch((val) => console.log("Exception caught 2222",val));

... ..... ..... ..... ..... ... ..... ..... ..... ... ...
Exception caught Exception333


#### Chaining then actions to a promise
Few things to observe in the next test:
* You can't start chained lines with .then in this Jupyter notebook. Need to put . on previous line
* then(...) need to return the next value to use

In [30]:
%%node
promiseA.then( (val) => {console.log("Resolved:",val); return val*2} ).
then( (val) => {console.log("Resolved 2222:",val);return val}).
then( (val) => {console.log("Resolved 3333:",val);return val}).
catch((val) => console.log("Exception caught",val))

... ... ...
Resolved: 777
Resolved 2222: 1554
Resolved 3333: 1554


#### async / await keywords

These provide an alternative way to create asynchronous programs.
An async function is a promise and can be used with promise attachments.

An await into an async function is a non-blocking wait for the statement (which could be a promise to complete).

They provide a way to define an asynchronous function in a more synchronous manner.


In [38]:
%%node
async function testAsync1(msg){
    await promiseA.then( (val) => {console.log("PromiseA:",val); return val*2} );
    return "777: " + msg
}

... ... ...


In [39]:
%%node
testAsync1("hello").then((val) => {console.log("Resolved 2222:",val);return val})

PromiseA: 777
Resolved 2222: 777: hello


In [73]:
%%node

// You need to declare the i variable with local scope else the same global i will be shared across the 2 async functions
// And the loops will not work as you expect
async function testAsync2(){
    for (var i = 0; i < 10; i++) {
        // With the await infront this will relinquish control until the loop cycle
        // Hence allowing other function to make progress
        await console.log("testAsync2", i)
    }
    return "testAsync2 finished"
}

async function testAsync3(){
    for (var i = 0; i < 10; i++) {
        await console.log("testAsync3", i)
    }
}

... ..... ..... ..... ..... ... ...
... ..... ..... ...


In [78]:
%%node
 testAsync2()
 testAsync3()


testAsync2 0
testAsync3 0
testAsync2 1
testAsync3 1
testAsync2 2
testAsync3 2
testAsync2 3
testAsync3 3
testAsync2 4
testAsync3 4
testAsync2 5
testAsync3 5
testAsync2 6
testAsync3 6
testAsync2 7
testAsync3 7
testAsync2 8
testAsync3 8
testAsync2 9
testAsync3 9
